In [1]:
import os, sys
print("Current folder:", os.getcwd())
sys.path.insert(0, os.path.abspath(".."))
print("Project root added:", os.path.abspath(".."))

Current folder: /home/jovyan/EEP-153---Project-1/notebooks
Project root added: /home/jovyan/EEP-153---Project-1


In [2]:
import logging
logging.getLogger("shelved_cache.persistent_cache").setLevel(logging.ERROR)

In [3]:
from src.indicators import INDICATORS
from src.wb_fetch import fetch_wb_data
from src.panel import build_panel

In [4]:
import wbdata
import pandas as pd
countries_list = wbdata.get_countries()
countries_df = pd.DataFrame(countries_list)
countries_df.columns

Index(['id', 'iso2Code', 'name', 'region', 'adminregion', 'incomeLevel',
       'lendingType', 'capitalCity', 'longitude', 'latitude'],
      dtype='object')

In [5]:
def region_value(x):
    if isinstance(x, dict):
        return x.get("value")
    return x
countries_df["region_value"] = countries_df["region"].apply(region_value)
countries_only = countries_df[countries_df["region_value"] != "Aggregates"].copy()
country_codes = countries_only["id"].tolist()
len(country_codes), countries_only[["id","name","region_value"]].head()

(217,
     id         name                                       region_value
 0  ABW        Aruba                         Latin America & Caribbean 
 2  AFG  Afghanistan  Middle East, North Africa, Afghanistan & Pakistan
 5  AGO       Angola                                Sub-Saharan Africa 
 6  ALB      Albania                              Europe & Central Asia
 7  AND      Andorra                              Europe & Central Asia)

In [6]:
df_countries = fetch_wb_data(
    INDICATORS,
    start_year=2000,
    end_year=2022,
    countries=country_codes,
    cache_path="../data/wb_cache_countries.csv")
df_countries.shape, df_countries["country"].nunique(), df_countries["country"].head(10)

((4991, 6),
 217,
 0    Aruba
 1    Aruba
 2    Aruba
 3    Aruba
 4    Aruba
 5    Aruba
 6    Aruba
 7    Aruba
 8    Aruba
 9    Aruba
 Name: country, dtype: object)

In [7]:
panel = build_panel(df_countries)
panel.shape, panel["country"].nunique(), panel.head()

((4991, 7),
 217,
   country  ag_export_share  food_prod_index  adult_mortality_male  \
 0   Aruba         0.228271              NaN               118.505   
 1   Aruba         0.102736              NaN               159.045   
 2   Aruba         0.069199              NaN               134.254   
 3   Aruba         0.104098              NaN               126.903   
 4   Aruba         0.076267              NaN               130.119   
 
    adult_mortality_female  year  export_dependent  
 0                  60.053  2022             False  
 1                  85.706  2021             False  
 2                  62.616  2020             False  
 3                  59.787  2019             False  
 4                  60.625  2018             False  )

In [8]:
panel[["ag_export_share","food_prod_index","adult_mortality_male","adult_mortality_female"]].isna().mean().sort_values()

adult_mortality_female    0.003005
adult_mortality_male      0.003005
food_prod_index           0.108595
ag_export_share           0.228010
dtype: float64